# Pink Noise Filtered Dirac Impulse
aka Pink Impulse

cf. Juho Liski, Aki Mäkivirta, Vesa Välimäki (2018): "Audibility of Loudspeaker Group-Delay Characteristics.", In *Proc. of 144th Audio Eng. Soc. Conv.*, Milano, #10008

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import signal
import soundfile as sf

In [ ]:
N = int(2**14+1)  # DFT length
kaiserbeta = 8.17
kaiserM = 4401 #4401 # final length of pink noise shaped impulse

k = np.arange(0,kaiserM)+N//2-kaiserM//2
w = signal.kaiser(kaiserM, kaiserbeta, sym=True)
W = np.arange(0, 2*np.pi, 2*np.pi/N)
print(W.shape)

phase = 0*np.pi/4

if True:  # PN_Lowpass
    H = (1/np.sqrt(np.abs(W))*1+0) * np.exp(1j*phase)
else:  # PN_Highass
    H = (np.sqrt(np.abs(W))*1+0) * np.exp(1j*phase)

H[0] = H[1]
if np.mod(N,2)==0:
    H[N//2+1:] = np.conjugate(np.flip(H[1:N//2]))
else:
    H[(N-1)//2+1:] = np.conjugate(np.flip(H[1:(N-1)//2+1])) 

plt.subplot(2,1,1)    
plt.plot(W,20*np.log10(np.abs(H)))
plt.grid(True)
plt.ylabel('H / dB')
plt.subplot(2,1,2)    
plt.plot(W,np.angle(H)*180/np.pi)
plt.grid(True)
plt.xlabel(r'$\Omega$');
plt.ylabel('arg(H) / rad')
plt.xlabel(r'$\Omega / rad$')
plt.ylim(-200,200)
plt.yticks(np.arange(-180,225,45));

In [ ]:
h = np.fft.ifft(H)
print('max imag in h:', np.max(np.abs(h.imag)))
h = np.roll(h.real,N//2)
h = h/np.max(np.abs(h))
hw = h[N//2-kaiserM//2:N//2+kaiserM//2+1] * w

plt.subplot(2,1,1)
plt.plot(20*np.log10(np.abs(h)), label='long rect win')
plt.plot(k,20*np.log10(np.abs(hw)), label='short kaiser win')
plt.grid(True)
plt.ylim(-200,0)
plt.ylabel('h[k] / dB')

plt.subplot(2,1,2)
plt.plot(k,h[N//2-kaiserM//2:N//2+kaiserM//2+1], label='long rect win')
plt.plot(k,hw, label='short kaiser win')
plt.grid(True)
plt.ylim(-1.1,1.1)
plt.xlabel('k')
plt.ylabel('h[k]');

In [ ]:
wav = np.column_stack((h, h))
wav = np.column_stack((hw, hw))
wav.shape
sf.write('../data/source-signals/PNDirac.wav', wav, 44100)